# Week 1 - January 23

## Monday 01/02

### WSL and X-session

I have left on-going tests last week of 22.

Trying to start a X-session on ubuntu 22.04 within this new version of WSL.

Done it on a sandbox distro at [Run gnome-session in Ubuntu LTS](/posts/2022-12-15-wsl2-fresh-pc.html#run-gnome-session-in-ubuntu-lts)

Now `gnome-shell` can be started from distro `ubuntu-x11` by calling `gnomeshell.sh`

## Tuesday 01/03

### other wsl stuff

**jupyter lab**

setup my `base_jupyter` env [jupyter lab in WSL](/posts/2022-12-15-wsl2-fresh-pc.html#jupyter-lab)

Now jupyter lab starts automatically with distro `ubuntu-22.04`

and can be restarted with `sudo systemctl restart jupyterlab` 

**[zotero](/posts/2022-12-15-wsl2-fresh-pc.html#zotero)**

can install and use it including sync (otherwise no interest)

## Wednesday 01/04

get back to [fastai lesson 8](/posts/2022-10-17-fastai-2022-courses.html#lesson-8---convolutions-cnns)

opened a ticket in wsl github : [WSL some icons in taskbar for linux applications are defaulting to Tux](https://github.com/microsoft/wslg/issues/944) (triaged to wslg)

## Friday 01/06

Working with collab filtering (to fill NaN), inspired by fastai [Lesson 8 - Practical Deep Learning for Coders 2022](https://www.youtube.com/watch?v=htiNBPxcXgo&t=1086s)

seems it takes 6x times longer to use collab_learner instead of directly embeddings

I need to understand why. I know why, it was just a mistake in calling `collab_learner(dls, n_factors, y_range=y_range)collab_learner(dls, n_factors, y_range=y_range)` (gorgot `y_range=`)

# Week 2 - January 23

## Tuesday 01/09

I have continued my experimentations with collaborative filtering and embedding matrices. With nice results. I jave played with parameters (y_range) and hyper parameters (wd). Will have to share with colleagues to get their feedback.

I have started to use collab_learner with NN. But I have to better understand what is behind. 

Continue to listen to this lesson 8. It is back to computer vision with convolution. I have less interest for these matters but I still like the way Jeremy uses Excel to illustrate all these concepts. Jeremy: the AI excel master.